In [1]:
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
# read videos
import pylab          # play video
from os import listdir
import cv2
# others
import os
from PIL import Image
import random
import matplotlib.pyplot as plt
# SINDy
import sys
sys.path.append("src")
import sindy_utils as sindy
import numpy as np


### Functions

In [2]:
def printProgress(epoch, batch_id, loss):
    """
    print progress of the training
    epoch: number
    batch_id: current batch_id
    accuracy:
    loss:
    
    """
    progress = '='* int((10. * (batch_id+1) / len(train_data)))
    progress += '>'
    if batch_id == 0:
        print('Train Epoche {}: {}% [{}]\t , loss: {:.6f}'.format(
            epoch+1, int(100. * (batch_id+1) / len(train_data)),progress, loss.item()), end='')
    else:
        print('\rTrain Epoche {}: {}% [{}]\t , loss: {:.6f}'.format(
            epoch+1, int(100. * (batch_id+1) / len(train_data)),progress, loss.item()), end='', flush = True)
        

def calculateLoss(network, params):
    """
    calculate the loss of autoencoder and SINDy combined. loss function of:
    
     O \   _________           ________  /  O
     .    |         | \  O  / |        |    .
     . -  | phi'(x) | -  O  - | phi(z) | -  .
     .    |_________| /  O  \ |________|    .
     O /                                 \  O
    x(t)                z(t)               xa(t)
    
    ||x-phi(z)||_2^2 + lam1 ||dx - (zGrad phi(z)) Theta(z^T) Xi||_2^2 + lam2 ||dz - Theta(z^T) Xi||_2^2 + lam3 ||Xi||_1
        decoder      +                   SINDy in dx                  +         SINDy in dz             +   SINDy sparsity
     
    dz = xGrad phi'(x) dx = (xGrad z) dx
    
    network: data of the network
    params: hyperparameters
    
    """
    dx = network['dx']
    dx_decode = network['dx_decode']
    dz = network['dz']
    dz_predict = network['dz_sindy']
    Xi_coeff = network['Xi']
    rec_loss = network['rec_loss']
    sindy_x_loss = torch.mean((dx-dx_decode)**2)
    sindy_z_loss = torch.mean((dz-dz_predict)**2)
    sparse_loss = torch.mean(torch.abs(Xi_coeff))
    
    # separate view of each loss
    separate_loss = []
    separate_loss.append(rec_loss)
    separate_loss.append(sindy_x_loss)
    separate_loss.append(sindy_z_loss)
    separate_loss.append(sparse_loss)
    
    tot_loss = (params['loss_weight_decoder'] * rec_loss
                + params['loss_weight_sindy_x'] * sindy_x_loss 
                + params['loss_weight_sindy_z'] * sindy_z_loss
                + params['loss_weight_sindy_regularization'] * sparse_loss)
                                                                                        
    return tot_loss, separate_loss


def calculateSindy(network, params):
    '''
    Calculate Theta(z), Xi and dz
    
    '''
    z = network['z'].detach().numpy()
    dz = network['dz'].detach().numpy()
    
    network['Theta'] = torch.from_numpy(sindy.sindy_library(z, params['poly_order'], include_sine=params['include_sine']))
    network['Xi'] = torch.from_numpy(sindy.sindy_fit(network['Theta'], dz, params['sindy_threshold']))
    dz_predict = torch.matmul(network['Theta'],network['Xi'])
    
    return dz_predict

### Define model parameters

In [3]:
params = {}

# autoencoder settings
params['number_epoch'] = 2                               # number of epochs
params['z_dim'] = 10                                     # number of coordinates for SINDy
params['batch_size'] = 8                                # batch size
params['lr_rate'] = 1e-2                                 # learning rate
params['weight_decay'] = 1e-8


# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_regularization'] = 1e-5

# SINDy parameters
params['sindy_threshold'] = 0.5 
params['poly_order'] = 2
params['include_sine'] = False

### Data processing

In [ ]:
# read the train videos in random order
file_names = []
for f in listdir('../../Videos/train/'):
    if f != 'high_res':
        file_names.append(f)

random.shuffle(file_names)

# define transform to tensor and resize to 1080x1920
normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])    # normalize around mean with sigma (std)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((1080, 1920))])

# read data to list and transform to tensor
train_data_tmp = []
train_data = []
train_idxOfNewVideo = []
count = 0
for f in file_names:
    # just for testing (save time)
    if count == 1:
        break
    count += 1
    vidcap = cv2.VideoCapture('../../Videos/train/' + f)
    success,imgR = vidcap.read()
    print('Read:',f)
    while success:
        imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)
        imgR_tensor = transform(imgR)
        train_data_tmp.append(imgR_tensor)
        success,imgR = vidcap.read()
        # make a batch
        if len(train_data_tmp) >= params['batch_size']:
            train_data.append(torch.stack(train_data_tmp))
            train_data_tmp = []
    train_idxOfNewVideo.append(len(train_data))
    print(len(train_data), len(train_data[0]), len(train_data[0][0]), len(train_data[0][0][0]), len(train_data[0][0][0][0]))


Read: St_near_timelapse_low.mp4


In [ ]:
# split into validation and training set
print(len(train_data), len(train_data[0]), len(train_data[0][0]), len(train_data[0][0][0]), len(train_data[0][0][0][0]))

validation_data = []
# take 10% of training set batches to validation set
nbr_batch = int(len(train_data)*0.1)
for i in range(0,nbr_batch):
    choose = random.randint(0, len(train_data)-1)
    element = train_data[choose]
    validation_data.append(element)
    train_data.pop(choose)

print('done:')
print(len(validation_data), len(validation_data[0]), len(validation_data[0][0]), len(validation_data[0][0][0]), len(validation_data[0][0][0][0]))
print(len(train_data), len(train_data[0]), len(train_data[0][0]), len(train_data[0][0][0]), len(train_data[0][0][0][0]))


In [ ]:
# read test videos
test_data = []
test_idxOfNewVideo = []

# define transform to tensor and resize to 1080x1920
normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])    # normalize around mean with sigma (std)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((1080, 1920))])

# read data to list and transform to tensor
count = 0
for f in listdir('../../Videos/test/'):
    if f != 'high_res':
        # just for testing (save time)
        if count == 1:
            break
        count += 1
        vidcap = cv2.VideoCapture('../../Videos/test/' + f)
        success,imgR = vidcap.read()
        print('Read:',f)
        while success:
            imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)
            imgR_tensor = transform(imgR)
            test_data.append(imgR_tensor)
            success,imgR = vidcap.read()
        test_idxOfNewVideo.append(len(test_data))
    
print(len(test_data), len(test_data[0]), len(test_data[0][0]), len(test_data[0][0][0]))


In [ ]:
# plot some frames and test prints
print(torch.min(train_data[0]), torch.max(train_data[0]))
print(train_data[0].permute(3,2,1,0).size())

# plot first frame per batch
for i in range(len(train_data)):
    if i%3 == 0:
        plt.figure()
    imgP = train_data[i][0].permute(1,2,0).detach().numpy()
    plt.subplot(1,3, i%3 + 1)
    plt.imshow(imgP)

               

### SINDy autoencoder

In [ ]:
# autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__() 
        self.encode = nn.Sequential(
            # encoder: N, 3, 1080, 1920
            nn.Conv2d(3, 16, 5), # N, 16, 1076, 1916
            nn.ReLU(),
            nn.Conv2d(16, 32, 5), # N, 32, 1072, 1912
            nn.ReLU(),
            nn.MaxPool2d((2,3), stride=(2,3)), # N, 32, 536, 637              -- pool --
            nn.Conv2d(32, 64, 8, stride=2, padding=1), # N, 64, 266, 316
            nn.ReLU(),
            nn.Conv2d(64, 96, 7), # N, 96, 260, 310
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2), # N, 96, 130, 155                      -- pool --
            nn.Conv2d(96, 128, 11), # N, 128, 120, 145
            nn.ReLU(),
            nn.Conv2d(128, 150, 4, stride=2, padding=1), # N, 150, 60, 72
            nn.ReLU(),
            nn.Conv2d(150, 200, 11), # N, 200, 50, 62
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), # N, 200, 25, 31                        -- pool --
            nn.Conv2d(200, 200, 9, stride=2, padding=1), # N, 200, 10, 13
            nn.ReLU(),
            nn.Conv2d(200, 200, 10), # N, 200, 1, 4
            nn.ReLU()
        )
        
        self.fc1 = nn.Linear(200*4,params['z_dim'])
        # Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
        # Input [-1, +1] -> use nn.Tanh    
        
        # note: encoder and decoder are not symmetric
        self.decode = nn.Sequential(
            nn.ConvTranspose2d(200, 200, 4), # N, 200, 4, 7
            nn.ReLU(),
            nn.ConvTranspose2d(200, 200, 9, stride=2, padding=2, output_padding=(1,0)), # N, 200, 12, 17
            nn.ReLU(),
            nn.ConvTranspose2d(200, 150, 8, stride=4, padding=(0,1), output_padding=(3,0)), # N, 150, 55, 70
            nn.ReLU(),
            nn.ConvTranspose2d(150, 128, 8), # N, 128, 62, 77
            nn.ReLU(),
            nn.ConvTranspose2d(128, 96, 6, stride=(2,3), padding=1), # N, 96, 126, 232
            nn.ReLU(),
            nn.ConvTranspose2d(96, 64, 7), # N, 64, 132, 238
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 8, stride=2, padding=(1,2), output_padding=(1,1)), # N, 32, 296, 479
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 5, stride=2, padding=1), # N, 16, 539, 959
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 5, stride=2, padding=1, output_padding=(1,1)), # N, 3, 1080, 1920
            nn.ReLU()
        )   
        
        self.fc2 = nn.Linear(params['z_dim'], 200*4)

    def forward(self, x, z, mode):
        '''
        x: input for encoder
        z: input for decoder
        mode: 
            'train' -> use encoded for decoder
            'test'  -> feed z in an get decoded
        
        '''
        if mode == 'train':
            #print('train mode')
            encoded = self.encode(x)
            encoded = encoded.view(-1,200*4)
            encoded = self.fc1(encoded)

            decoded = self.fc2(encoded)
            decoded = decoded.view(-1,200,1,4)
            decoded = self.decode(decoded)            
        else:
            #print('test mode')
            encoded = torch.zeros(1)

            decoded = self.fc2(z)
            decoded = decoded.view(-1,200,1,4)
            decoded = self.decode(decoded)
        
        return encoded, decoded


In [ ]:
# load model
autoencoder_path = 'results/Autoencoder_#epoch_v1.pt'
if os.path.isfile(autoencoder_path):
    autoencoder = torch.load(autoencoder_path)
    print('loaded autoencoder', encoder_path)
else:
    autoencoder = Autoencoder()
    print('loaded new autoencoder')

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# optimization technique
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=params['lr_rate'], weight_decay=params['weight_decay'])

# to save network data
network = {}

# training function
outputs = []
def train(epoch):
    '''
    training function for the autoencoder

    '''
    for batch_id,img in enumerate(train_data):      
        encode_tensor, recon_tensor = autoencoder(img, 0, mode='train')
        network['rec_loss'] = criterion(recon_tensor, img)
    
        # x, z is current batch_id, dx, dz is next one (in else we take dz as current and compare with x from before)
        if batch_id == 0:
            total_loss = network['rec_loss']       
            network['z'] = encode_tensor
        else:
            network['dx'] = img
            network['dz'] = encode_tensor
            network['dz_sindy'] = calculateSindy(network, params).float()
            _, network['dx_decode'] = autoencoder(0, network['dz_sindy'], mode='test')
            total_loss, _ = calculateLoss(network, params)            # total loss with SINDy
            # now advance one step
            network['z'] = network['dz']
                
        # optimization and backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        # print progress
        printProgress(epoch, batch_id, total_loss)
        if batch_id == 2:
            break
    print('\n')
    outputs.append((epoch, img, recon_tensor))
    
# evaluation function
evaluated_dict = {}
ae_loss = []
sindy_loss = []
def evaluate():
    '''
    evaluation of the training by it's loss

    '''
    autoencoder.eval()
    ae_lossE = 0
    sindy_lossE = 0
    for i, img in enumerate(validation_data):
        # an other video sequence
        if i % params['batch_size'] == 0:
            # x, z are at the current time
            evaluated_dict['z'], evaluated_dict['x'] = autoencoder(img, 0, mode='train')
            eval_theta = torch.from_numpy(sindy.sindy_library(evaluated_dict['z'].detach().numpy(), params['poly_order'], include_sine=params['include_sine']))
            evaluated_dict['dz_sindy'] = torch.matmul(eval_theta,network['Xi']).float()
            _, evaluated_dict['dx_sindy'] = autoencoder(0, evaluated_dict['dz_sindy'], mode='test')
            # autoencoder loss
            ae_lossE += F.mse_loss(evaluated_dict['x'], img)
        else:
            # sindy loss
            sindy_lossE += F.mse_loss(evaluated_dict['dx_sindy'], img)
            evaluated_dict['z'], evaluated_dict['x'] = autoencoder(img, 0, mode='train')            
            eval_theta = torch.from_numpy(sindy.sindy_library(evaluated_dict['z'].detach().numpy(), params['poly_order'], include_sine=params['include_sine']))
            evaluated_dict['dz_sindy'] = torch.matmul(eval_theta,network['Xi']).float()
            _, evaluated_dict['dx_sindy'] = autoencoder(0, evaluated_dict['dz_sindy'], mode='test')
            ae_lossE += F.mse_loss(evaluated_dict['x'], img)
    
    # append average loss of this epoch
    ae_loss.append(ae_lossE/len(validation_data))
    sindy_loss.append(sindy_lossE(len(validation_data)))

for epoch in range(params['number_epoch']):
    train(epoch)
    evaluate()


In [ ]:
# example prints
torch.save(autoencoder, 'results/Autoencoder_2epoch_v1.pt')


In [ ]:
# testing function
def test():
    pass


### Prediction and plots

In [ ]:
# SINDy prediction --> split up model that I can plug in z
t = np.arange(0,20,.01)
z_sim = sindy.sindy_simulate(network['z'], t, network['Xi'], params['poly_order'], params['include_sine'])

In [ ]:
# plots
print(network['Theta'].shape, network['Xi'].shape)
#print(network['Xi'])

print('reconstruction loss', categorie_loss[0],'\n sindy x loss', categorie_loss[1], 
      '\n sindy z loss', categorie_loss[2], '\n Xi sparsity loss', categorie_loss[3])
print(network['rec_loss'])

for i in range(0, params['number_epoch']):
    plt.figure()
    realImg = outputs[i][1].permute(0,2,3,1).detach().numpy()
    reconImg = outputs[i][2].permute(0,2,3,1).detach().numpy()
    for i, item in enumerate(realImg):
        if i >=4: break
        plt.subplot(2,4,i+1)
        plt.imshow(item)
    for i, item in enumerate(reconImg):
        if i >=4: break
        plt.subplot(2,4,4+i+1)
        plt.imshow(item)    
